Exercises I

1. Run python -m pip install pymysql from your terminal to install the mysql client (any folder is fine)

2. cd into your exercises folder for this module and run echo env.py >> .gitignore

3. Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [1]:
import pandas as pd 
import numpy as np
from env import host, user, password

In [2]:
def get_db_url(database, user=user, password=password, host=host):
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

4. Use your function to obtain a connection to the employees database.

In [3]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', get_db_url('employees'))


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


Once you have successfully run a query:

a. Intentionally make a typo in the database url. What kind of error message do you see?

b. Intentionally make an error in your SQL query. What does the error message look like?

In [28]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', get_db_url('employe'))
# a. I get operational errors.

OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'innis_1664'@'%' to database 'employe'")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [29]:
pd.read_sql('SELECT * FRO employ LIMIT 5 OFFSET 50', get_db_url('employees'))
# b. I get programing errors

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FRO employ LIMIT 5 OFFSET 50' at line 1")
[SQL: SELECT * FRO employ LIMIT 5 OFFSET 50]
(Background on this error at: https://sqlalche.me/e/14/f405)

6. Read the employees and titles tables into two separate DataFrames.

In [4]:
employees = pd.read_sql('SELECT * FROM employees', get_db_url('employees'))
titles = pd.read_sql('SELECT * FROM titles ', get_db_url('employees'))

7. How many rows and columns do you have in each DataFrame? Is that what you expected?

In [27]:
employees.shape #300024 rows and 6 columns 

(300024, 6)

In [16]:
titles.shape # 443308 rows and 4 columns 

(443308, 4)

8. Display the summary statistics for each DataFrame.

In [28]:
employees.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [29]:
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


9. How many unique titles are in the titles DataFrame?

In [31]:
titles.head()
titles.title.unique()
# 7 unique titles 

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader', 'Manager'], dtype=object)

10. What is the oldest date in the to_date column?

In [32]:
titles.to_date.min() # 1985-03-01

datetime.date(1985, 3, 1)

11. What is the most recent date in the to_date column?

In [64]:
titles.to_date.max() # 9999-01-01

datetime.date(9999, 1, 1)

Exercises II

1. Copy the users and roles DataFrames from the examples above.

In [32]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [34]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


2. What is the result of using a right join on the DataFrames?

In [35]:
right_join = users.merge(roles, left_on='role_id', right_on='id', how='right', indicator= True)
right_join

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


3. What is the result of using an outer join on the DataFrames?

In [36]:
outer_join = users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator= True)
outer_join

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [37]:
users_role_id_dropped = users.drop(columns='role_id')
users_role_id_dropped.merge(roles, on = 'id')

# It's a mistake, adam is a commenter which isn't true 


,id,name_x,name_y
0,1,bob,admin
1,2,joe,author
2,3,sally,reviewer
3,4,adam,commenter


5. Load the mpg dataset from PyDataset.

In [38]:
from pydataset import data 
mpg = data('mpg')

6. Output and read the documentation for the mpg dataset.

In [39]:
data('mpg', show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




7. How many rows and columns are in the dataset?

In [40]:
mpg.shape #234 rows and 11 columns

(234, 11)

8. Check out your column names and perform any cleanup you may want on them.

In [41]:
mpg.columns

Index(['manufacturer', 'model', 'displ', 'year', 'cyl', 'trans', 'drv', 'cty',
       'hwy', 'fl', 'class'],
      dtype='object')

In [42]:
mpg.rename(columns= {'displ':'displacement', 'cyl':'cylinders','trans':'transmission','drv':'drivetrain', 'cty':'city', 'hwy':'highway', 'fl':'fuel'}, inplace=True)
mpg.head()

,manufacturer,model,displacement,year,cylinders,transmission,drivetrain,city,highway,fuel,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


9. Display the summary statistics for the dataset.

In [24]:
mpg.describe()

,displacement,year,cylinders,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


10. How many different manufacturers are there?

In [45]:
mpg.manufacturer.nunique() # 15 unique manufacturers

15

11. How many different models are there?

In [26]:
mpg.model.unique().size #38 different models

38

12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [49]:
mpg['mileage_difference'] = mpg.highway - mpg.city
mpg.head()

,manufacturer,model,displacement,year,cylinders,transmission,drivetrain,city,highway,fuel,class,average_mileage,is_automatic,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5,True,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0,False,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5,False,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5,True,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0,True,10


13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [47]:
mpg['average_mileage'] = mpg[['city', 'highway']].mean(axis=1).round(1)
mpg.head()

,manufacturer,model,displacement,year,cylinders,transmission,drivetrain,city,highway,fuel,class,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0


14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [51]:
mpg['is_automatic'] = np.where(mpg['transmission'].str.contains('auto'), True , False)
mpg.head()

,manufacturer,model,displacement,year,cylinders,transmission,drivetrain,city,highway,fuel,class,average_mileage,is_automatic,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,23.5,True,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,25.0,False,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,25.5,False,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,25.5,True,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,21.0,True,10


15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [50]:
mpg.groupby('manufacturer').average_mileage.mean().sort_values(ascending = False).round(2)
# Honda has the best mpg on average

manufacturer
honda         28.50
volkswagen    25.07
hyundai       22.75
subaru        22.43
audi          22.03
toyota        21.72
pontiac       21.70
nissan        21.35
chevrolet     18.45
ford          16.68
mercury       15.62
jeep          15.56
dodge         15.54
lincoln       14.17
land rover    14.00
Name: average_mileage, dtype: float64

16. Do automatic or manual cars have better miles per gallon?

In [52]:
mpg.groupby('is_automatic').average_mileage.mean()
# Manual has better average mpg

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

Exercises III

1. Use your get_db_url function to help you explore the data from the chipotle database.

In [108]:
chipotle = pd.read_sql('SELECT * FROM orders', get_db_url('chipotle'))
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


2. What is the total price for each order?

In [109]:
chipotle['item_price'] = chipotle.item_price.str.strip('$')
chipotle['item_price'] = pd.to_numeric(chipotle.item_price)
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39
1,2,1,1,Izze,[Clementine],3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98


In [123]:
chipotle.groupby('order_id').item_price.sum()

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

3. What are the most popular 3 items?

In [119]:
chipotle.groupby('item_name').quantity.sum().sort_values(ascending=False).head(3)

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

4. Which item has produced the most revenue?

In [134]:
chipotle.groupby('item_name').item_price.sum().sort_values(ascending=False).head(1)

item_name
Chicken Bowl    7342.73
Name: item_price, dtype: float64

5. Join the employees and titles DataFrames together.

In [6]:
employees_and_titles = pd.merge(employees, titles, left_on='emp_no', right_on='emp_no', how='left')
employees_and_titles.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


6. For each title, find the hire date of the employee that was hired most recently with that title.

In [7]:
employees_and_titles.groupby('title').hire_date.max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)



In [8]:
departments = pd.read_sql('SELECT * FROM departments', get_db_url('employees'))
dept_emp = pd.read_sql('SELECT * FROM dept_emp', get_db_url('employees'))


In [9]:
employees_titles_dept_emp = pd.merge(employees_and_titles, dept_emp, left_on='emp_no', right_on='emp_no', how='left')

In [10]:
employees_db_merged = pd.merge(employees_titles_dept_emp, departments, left_on='dept_no', right_on='dept_no', how='left')
employees_db_merged.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date_x,to_date_x,dept_no,from_date_y,to_date_y,dept_name
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,1986-06-26,9999-01-01,Development
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01,d007,1996-08-03,9999-01-01,Sales
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01,d004,1995-12-03,9999-01-01,Production
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01,d004,1986-12-01,9999-01-01,Production
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01,d004,1986-12-01,9999-01-01,Production


In [12]:
pd.crosstab(employees_db_merged.dept_name, employees_db_merged.title, margins=True)

title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader,All
dept_name,,,,,,,,
Customer Service,298,2362,4,2027,13925,16150,309,35075
Development,7769,58135,2,49326,1247,1424,7683,125586
Finance,0,0,2,0,12139,13929,0,26070
Human Resources,0,0,2,0,12274,14342,0,26618
Marketing,0,0,2,0,13940,16196,0,30138
Production,6445,49649,4,42205,1270,1478,6557,107608
Quality Management,1831,13852,4,11864,0,0,1795,29346
Research,378,2986,2,2570,11637,13495,393,31461
Sales,0,0,2,0,36191,41808,0,78001
